<a href="https://colab.research.google.com/github/VicenteJFonseca/previs-o_pre-o_carros/blob/main/previs%C3%A3o_pre%C3%A7os_carros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fonte dados: https://www.kaggle.com/datasets/vagnerbessa/average-car-prices-bazil


---



# Importando Dados e Bibliotecas Python

In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
fipe22 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/fipe_2022.csv", delimiter = ",")

# Visualização dos Dados

In [4]:
fipe22.head()

,year_of_reference,month_of_reference,fipe_code,authentication,brand,model,fuel,gear,engine_size,year_model,avg_price_brl,age_years
0,2022,January,038001-6,vwmrywl5qs,Acura,NSX 3.0,Gasoline,manual,3.0,1995,43779.0,28
1,2022,January,038001-6,t9mt723qhz,Acura,NSX 3.0,Gasoline,manual,3.0,1994,42244.0,29
2,2022,January,038001-6,tr5wv4z21g,Acura,NSX 3.0,Gasoline,manual,3.0,1993,40841.0,30
3,2022,January,038001-6,s2xxsjz3mt,Acura,NSX 3.0,Gasoline,manual,3.0,1992,39028.0,31
4,2022,January,038001-6,rtm9gj7zk8,Acura,NSX 3.0,Gasoline,manual,3.0,1991,35678.0,32


In [5]:
fipe22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290275 entries, 0 to 290274
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   year_of_reference   290275 non-null  int64  
 1   month_of_reference  290275 non-null  object 
 2   fipe_code           290275 non-null  object 
 3   authentication      290275 non-null  object 
 4   brand               290275 non-null  object 
 5   model               290275 non-null  object 
 6   fuel                290275 non-null  object 
 7   gear                290275 non-null  object 
 8   engine_size         290275 non-null  float64
 9   year_model          290275 non-null  int64  
 10  avg_price_brl       290275 non-null  float64
 11  age_years           290275 non-null  int64  
dtypes: float64(2), int64(3), object(7)
memory usage: 26.6+ MB


## Aplicando Pandas Profiling

In [6]:
%pip install -U pandas-profiling
from pandas_profiling import ProfileReport
profile = ProfileReport(fipe22)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.7/328.7 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.5/679.5 KB 50.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 90.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 KB 30.4 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=f9ab3d0a15041fc779b0cf75f87d4cbda192a18202f2a83f16ade2e32b5de439
  Stored in directory: /root/.cache/pip/wheels/23/14/6e/4be5bfeeb027f4939a01764b48edd5996acf574b0913fe5243
Successfully built htmlmin
  Attempting uninstall: typeguard
    Found existing installation: typeguard 2.7.1
    Unin

In [ ]:
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

# Pré-processamento dos dados

## processamento da variável 'brand'

In [21]:
fipe22.brand.value_counts(normalize=True)

Other Brands       0.106416
GM - Chevrolet     0.097077
VW - VolksWagen    0.096949
Fiat               0.091510
Ford               0.083197
Mercedes-Benz      0.060184
Renault            0.052343
Audi               0.041919
Peugeot            0.038350
Toyota             0.035515
Mitsubishi         0.035077
BMW                0.034657
Citroën            0.030247
Hyundai            0.025786
Land Rover         0.024022
Nissan             0.023719
Porsche            0.023426
Volvo              0.020908
Honda              0.019671
Kia Motors         0.018855
Subaru             0.014255
Suzuki             0.013322
Jeep               0.012595
Name: brand, dtype: float64

In [22]:
brand = fipe22.brand.value_counts(normalize=True)

In [23]:
brand

Other Brands       0.106416
GM - Chevrolet     0.097077
VW - VolksWagen    0.096949
Fiat               0.091510
Ford               0.083197
Mercedes-Benz      0.060184
Renault            0.052343
Audi               0.041919
Peugeot            0.038350
Toyota             0.035515
Mitsubishi         0.035077
BMW                0.034657
Citroën            0.030247
Hyundai            0.025786
Land Rover         0.024022
Nissan             0.023719
Porsche            0.023426
Volvo              0.020908
Honda              0.019671
Kia Motors         0.018855
Subaru             0.014255
Suzuki             0.013322
Jeep               0.012595
Name: brand, dtype: float64

In [24]:
df_brand = pd.DataFrame(brand)

In [25]:
df_brand

,brand
Other Brands,0.106416
GM - Chevrolet,0.097077
VW - VolksWagen,0.096949
Fiat,0.091510
Ford,0.083197
Mercedes-Benz,0.060184
Renault,0.052343
Audi,0.041919
Peugeot,0.038350
Toyota,0.035515


In [26]:
df_brand[df_brand["brand"] < 0.01]

,brand


In [27]:
several_brands = df_brand[df_brand["brand"] < 0.01].index.tolist()

In [28]:
fipe22['brand'] = fipe22['brand'].replace( several_brands, 'Others Brands')

In [29]:
fipe22['brand'].value_counts(normalize = True)

Other Brands       0.106416
GM - Chevrolet     0.097077
VW - VolksWagen    0.096949
Fiat               0.091510
Ford               0.083197
Mercedes-Benz      0.060184
Renault            0.052343
Audi               0.041919
Peugeot            0.038350
Toyota             0.035515
Mitsubishi         0.035077
BMW                0.034657
Citroën            0.030247
Hyundai            0.025786
Land Rover         0.024022
Nissan             0.023719
Porsche            0.023426
Volvo              0.020908
Honda              0.019671
Kia Motors         0.018855
Subaru             0.014255
Suzuki             0.013322
Jeep               0.012595
Name: brand, dtype: float64

## Seleção de Variáveis

In [30]:
fipe22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290275 entries, 0 to 290274
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   year_of_reference   290275 non-null  int64  
 1   month_of_reference  290275 non-null  object 
 2   fipe_code           290275 non-null  object 
 3   authentication      290275 non-null  object 
 4   brand               290275 non-null  object 
 5   model               290275 non-null  object 
 6   fuel                290275 non-null  object 
 7   gear                290275 non-null  object 
 8   engine_size         290275 non-null  float64
 9   year_model          290275 non-null  int64  
 10  avg_price_brl       290275 non-null  float64
 11  age_years           290275 non-null  int64  
dtypes: float64(2), int64(3), object(7)
memory usage: 26.6+ MB


In [43]:
price_car = fipe22.drop(['year_of_reference','month_of_reference','fipe_code','year_model','authentication','model'], axis = 1)

In [44]:
price_car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290275 entries, 0 to 290274
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   brand          290275 non-null  object 
 1   fuel           290275 non-null  object 
 2   gear           290275 non-null  object 
 3   engine_size    290275 non-null  float64
 4   avg_price_brl  290275 non-null  float64
 5   age_years      290275 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 13.3+ MB


In [49]:
avg_price = pd.get_dummies(data = price_car, columns = ['brand','fuel','gear'])

In [50]:
avg_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290275 entries, 0 to 290274
Data columns (total 31 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   engine_size            290275 non-null  float64
 1   avg_price_brl          290275 non-null  float64
 2   age_years              290275 non-null  int64  
 3   brand_Audi             290275 non-null  uint8  
 4   brand_BMW              290275 non-null  uint8  
 5   brand_Citroën          290275 non-null  uint8  
 6   brand_Fiat             290275 non-null  uint8  
 7   brand_Ford             290275 non-null  uint8  
 8   brand_GM - Chevrolet   290275 non-null  uint8  
 9   brand_Honda            290275 non-null  uint8  
 10  brand_Hyundai          290275 non-null  uint8  
 11  brand_Jeep             290275 non-null  uint8  
 12  brand_Kia Motors       290275 non-null  uint8  
 13  brand_Land Rover       290275 non-null  uint8  
 14  brand_Mercedes-Benz    290275 non-nu